In [20]:
# Step 1: Install required packages
!pip install pyngrok diffusers torch torchvision --quiet

# Step 2: Import necessary libraries
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import torch
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi import Body
from pydantic import BaseModel
from diffusers import StableDiffusionPipeline
from typing import Optional

# Step 3: Set your ngrok auth token
ngrok.set_auth_token('AUTH_TOKEN')  # Replace with your ngrok token

# Step 4: Load the Stable Diffusion model from Google Drive
from google.colab import drive
drive.mount('/content/drive')

model_path = "/content/drive/My Drive/stable_diffusion_model"  # Adjust this path if necessary

def load_model():
    pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
    pipe = pipe.to("cuda")  # Send the model to GPU for faster inference
    return pipe

pipe = load_model()

# Step 5: Set up FastAPI app
app = FastAPI()

@app.get("/")
async def read_root():
    return {"message": "Welcome to the Stable Diffusion FastAPI server!"}

# Step 6: Define the request model
class GenerateRequest(BaseModel):
    prompt: str
    guidance_scale: Optional[float] = 7.5

@app.post("/generate")
async def generate_image(request: GenerateRequest):
    try:
        # Generate an image based on the prompt
        image = pipe(request.prompt, guidance_scale=request.guidance_scale).images[0]

        # Save the image to a file
        image.save("generated_image.png")

        return JSONResponse(content={"message": "Image generated successfully!", "image_path": "generated_image.png"})
    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)

# Step 7: Start ngrok tunnel and run the server
public_url = ngrok.connect(8000)
print(f"FastAPI public URL: {public_url}")

# Step 8: Run the server in the background
nest_asyncio.apply()  # Apply nest_asyncio to avoid runtime errors
uvicorn.run(app, host="0.0.0.0", port=8000)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

FastAPI public URL: NgrokTunnel: "https://bb3a-34-82-195-27.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [9790]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     103.111.134.56:0 - "POST /generate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9790]
